Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import pymystem3 as Mystem
import xgboost as xgb
import nltk
nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest , chi2
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV

from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, randint
import os
import re
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Места расположения папок с моделью BERT и исходными данными
DATA_PATH = "C:\\Users\ivan\\YandexDisk\\DS\\Project_11"

In [9]:
df_text = pd.read_csv(os.path.join(DATA_PATH,'toxic_comments.csv'))
df_text = df_text[:100]
# Делим текст тестовую
#df_text = df_text[:300]


In [4]:
df_text.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
print('Количество строк датасета', len(df_text))

Количество строк датасета 1000


In [6]:
# Проверим по некоторым сторокам какой там текст
print(df_text['text'][0],'\n')
print(df_text['text'][2],'\n')


Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27 

Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info. 



In [7]:
# Загружаем библиотеку
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Делаем из текста леммы
def text_prepare(row):
    # выделяем текст из строки, преобразуем, выделяем лемму, удаляем лишние символы
    text = row['text']   
    doc = nlp(text)   
    out_text= " ".join([token.lemma_ for token in doc ])      
    return re.sub('[^a-zA-Z]', ' ', out_text)

In [13]:
# Векторизуем токены
def vectorizer_func(data):
    # Выделяем корпус с леммами
    corpus = np.array(data.apply(text_prepare , axis=1)).astype('U')
    
    stopwords = set(nltk_stopwords.words('english'))
    target = data['toxic']
    
    # Делим корпус текста на train и test
    X_train , X_test , y_train , y_test  = train_test_split(corpus , target , test_size = 0.4 , random_state = 12)    
    
    
    vectorizer = TfidfVectorizer(stop_words = stopwords , ngram_range=(1,2))
    vect_tf_idf_train = vectorizer.fit_transform(X_train)
    vect_tf_idf_test = vectorizer.transform(X_test)
    return {'X_train':vect_tf_idf_train, 
            'X_test': vect_tf_idf_test , 
            'y_train': y_train , 
            'y_test': y_test }


In [14]:
vectorizer_func(df_text)

<60x3195 sparse matrix of type '<class 'numpy.float64'>'
	with 3988 stored elements in Compressed Sparse Row format>

In [ ]:
df_corpus = pd.DataFrame(data = corpus , columns = ['lemma_text'])
df = pd.concat([df_text , df_corpus] , axis = 1)
df.to_csv('dataframe_with_lemma.csv')

In [4]:
corpus = pd.read_csv(DATA_PATH +'\dataframe_with_lemma.csv' )
corpus = corpus['lemma_text'].values
corpus.shape

(159571,)

In [3]:
df_text =  pd.read_csv(DATA_PATH +'\dataframe_with_lemma.csv' )

In [4]:
# Векторизуем токены
def vectorizer_func(corpus):
    # Выделяем корпус с леммами
    #corpus = np.array(data.apply(text_prepare , axis=1)).astype('U')
    
    stopwords = set(nltk_stopwords.words('english'))
    target = df_text['toxic']
    
    # Делим корпус текста на train и test
    X_train , X_test , y_train , y_test  = train_test_split(corpus , target , test_size = 0.3 , random_state = 12)    
    
    
    vectorizer = TfidfVectorizer(stop_words = stopwords , ngram_range=(1,3) )
    vect_tf_idf_train = vectorizer.fit_transform(X_train)
    vect_tf_idf_test = vectorizer.transform(X_test)
    return {'X_train':vect_tf_idf_train, 
            'X_test': vect_tf_idf_test , 
            'y_train': y_train , 
            'y_test': y_test }

# 2. Обучение

In [5]:
%%time
# Записываем признаки и таргеты в переменные
train_dict = vectorizer_func(df_text['lemma_text'])



Wall time: 49.6 s


In [6]:
X_train = train_dict['X_train']
X_test = train_dict['X_test']
y_train = train_dict['y_train']
y_test = train_dict['y_test']

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((111699, 4671793), (47872, 4671793), (111699,), (47872,))

### Logistic regression CV

#### default parameters

In [77]:
%%time
# Загружаем модель и считаем f1 на кроссвалидации
lr_model = LogisticRegression(class_weight = 'balanced' ,  random_state = 12, C=1 , max_iter = 500)

# Выбираем лучшие признаки
selector = SelectKBest(chi2, k=50000)


clf_selected = make_pipeline(selector , lr_model)
scores = cross_val_score(clf_selected , X_train , y_train , scoring ='f1' , cv = 6)

Wall time: 2min 6s


In [78]:
print('Logistic regression Cross validation F1, default parameters:', scores.mean())

Logistic regression Cross validation F1, default parameters: 0.7626201090888548


#### Parameters tuning

In [75]:
%%time
lr_model = LogisticRegression(solver='lbfgs', 
                             # tol=1e-2, 
                              max_iter=500,
                              class_weight = 'balanced' ,
                              random_state = 12)
# параметры для поиска
distributions = {'lr_model__C':range(10,32,2) ,
                'selector__k':[50000 , 75000 , 100000]}

# Выбираем лучшие признаки
selector = SelectKBest(chi2)

clf_selected = Pipeline([('selector' , selector) , ('lr_model' , lr_model)])

clf = RandomizedSearchCV(clf_selected, distributions , random_state=12 , cv = 5 , scoring ='f1')
search = clf.fit(X_train, y_train)


Wall time: 22min 31s


In [76]:
print("Лучшие параметры:")
print(search.best_params_ )
print()

print("Лучший F1:")
print(search.best_score_)

({'selector__k': 75000, 'lr_model__C': 26}, 0.7690147378311532)

In [ ]:
# Сохраняем лучшую модель  и параметры
best_lr_model = search.best_estimator_

best_lr_params = search.best_params_ 

### XGBoost CV

#### default parameters

In [14]:
# Записываем данные
dtrain = xgb.DMatrix(X_train, label= y_train)
dtest  = xgb.DMatrix(X_test, label= y_test)


In [9]:
param = {'max_depth': 5, 'learning_rate': 1 }


In [25]:
# Загружаем модель , отбираем признаки
xgb_model = xgb.XGBClassifier( random_state=12)
selector = SelectKBest(chi2, k=10000)

clf_selected = Pipeline([('selector', selector) , ('xgb_model' , xgb_model)])

scores = cross_val_score(clf_selected , X_train , y_train , scoring ='f1' , cv = 5)

In [26]:
print('XGBoost Cross validation F1, default parameters:', scores.mean())

XGBoost Cross validation F1, default parameters: 0.7262231672170852


#### Parameters search

In [ ]:
%%time 
xgb_model = xgb.XGBClassifier(random_state=12)

# Параметры для поиска
param_grid = {'xgb_model__max_depth': [ 20 , 30, 40],
              'xgb_model__learning_rate': [0.2, 0.3, 0.4 ],
             #'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             #'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             #'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             #'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
             'xgb_model__gamma': [ 0.5, 1.0 , 1.5],
             #'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
              'xgb_model__n_estimators': [150 , 200, 230] , 
             'selector__k': [75000 , 85000 , 100000]}

# Выбираем признаки, 
selector = SelectKBest(chi2)
xgb_selected =Pipeline([('selector' , selector) , ('xgb_model' , xgb_model)])

xgb_search = RandomizedSearchCV(xgb_selected, param_grid, n_iter=10,
                            n_jobs=-1, cv=4,
                            scoring='f1', refit=True, random_state=12 , verbose = 2)
xgb_search.fit(X_train , y_train)

In [ ]:
xgb_search.cv_results_

In [9]:
print("Лучшие параметры:")
print(xgb_search.best_params_ )
print()

print("Лучший F1:")
print(xgb_search .best_score_)

Лучшие параметры:
{'xgb_model__n_estimators': 150, 'xgb_model__max_depth': 20, 'xgb_model__learning_rate': 0.2, 'xgb_model__gamma': 1.5, 'selector__k': 75000}

Лучший F1:
0.7575270360431507


In [10]:
# Сохраняем лучшую модель и лучшие параметры
best_xgb_model = xgb_search.best_estimator_

best_xgb_params = xgb_search.best_params_

### Logistic regression TEST

In [60]:
# Определяем значения на тестовых данных на лучшей модели
y_pred = best_lr_model.predict(X_test)


In [61]:
print('Logistic regression Test, F1:', f1_score(y_test , y_pred))

Pipeline(memory=None,
         steps=[('selector',
                 SelectKBest(k=50000,
                             score_func=<function chi2 at 0x0000002F290B3948>)),
                ('lr_model',
                 LogisticRegression(C=20, class_weight='balanced', dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=500,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=12,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

### XGBoost TEST

In [11]:
# Определяем значения на тестовых данных на лучшей модели
y_pred = best_xgb_model.predict(X_test)

In [12]:
print('XGBoost Test, F1-score:', f1_score(y_test , y_pred))

XGBoost Test, F1: 0.7824639289678136


# 3. Выводы

- Текст датасета лемматизирован библотекой spacy. С помощью регульрных выражений в строках оставлены только буквы и слова.
- После лемматизации корпус текста преобразован в матрицу со значениями TfIdf. Для создания признаков были ипользованы униграммы, биграммы и триграммы. В дальнейшем будут отобраны лучшие признаки.
- Для моделирования применены Logistic Regression и XFBoost. Рандомизированным поиском были определены оптимальные параметры моделей, а также оптимальное количество признаков. 
- На кроссвалидации с поиском параметров лучшие значения показала модель

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны